# Лабораторная работа №5
## Метрики качества классификации

Для оценки классификаторов используем другой набор данных - https://archive.ics.uci.edu/ml/datasets/SPECT+Heart

Данные отобображают зависимость между нормальным/аномальным состоянием сердца пациента и результатами его однофотонной эмиссионной компьютерной томографии. 
Соответственно этот набор данных имеет всего два класса: 1 и 0 (норма, аномалия), благодаря этому мы сможем оценить бинарную классификацию.

### Текст программы:

In [30]:
import pandas as pnd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


class Data:
    def __init__(self, data, train_size):
        self.x = data.values[:, 1:]
        self.y = data.values[:, 0]
        self.train_size = train_size
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(self.x, self.y,
                                                                                train_size=train_size)
        self.y_names = set(self.y)


def calculate_metrics(data, gnb, tree):
    scoring = ['accuracy', 'neg_log_loss', 'roc_auc']

    gnb_scores = cross_validate(gnb, data.x, data.y, scoring=scoring,
                                cv=5, return_train_score=False)

    tree_scores = cross_validate(tree, data.x, data.y, scoring=scoring,
                                 cv=5, return_train_score=False)

    gnb_mean, gnb_std = gnb_scores['test_accuracy'].mean(), gnb_scores['test_accuracy'].std()
    tree_mean, tree_std = tree_scores['test_accuracy'].mean(), tree_scores['test_accuracy'].std()
    print('Accuracy for GaussianNB: %0.3f (+/- %0.2f)\n'
          'Accuracy for RandomTree: %0.3f (+/- %0.2f)\n' % (gnb_mean, gnb_std, tree_mean, tree_std))

    gnb_mean, gnb_std = gnb_scores['test_neg_log_loss'].mean(), gnb_scores['test_neg_log_loss'].std()
    tree_mean, tree_std = tree_scores['test_neg_log_loss'].mean(), tree_scores['test_neg_log_loss'].std()
    print('Logarithmic loss for GaussianNB: %0.3f (+/- %0.2f)\n'
          'Logarithmic loss for RandomTree: %0.3f (+/- %0.2f)\n' % (gnb_mean, gnb_std, tree_mean, tree_std))

    gnb_mean, gnb_std = gnb_scores['test_roc_auc'].mean(), gnb_scores['test_roc_auc'].std()
    tree_mean, tree_std = tree_scores['test_roc_auc'].mean(), tree_scores['test_roc_auc'].std()
    print('Area under ROC curve for GaussianNB: %0.3f (+/- %0.2f)\n'
          'Area under ROC curve for RandomTree: %0.3f (+/- %0.3f)\n' % (gnb_mean, gnb_std, tree_mean, tree_std ))


def confus_matrix(data, gnb, tree):
    gnb_matrix = confusion_matrix(data.test_y, gnb.predict(data.test_x))
    print("Confusion matrix for GaussianNB")
    print(gnb_matrix)

    tree_matrix = confusion_matrix(data.test_y, tree.predict(data.test_x))
    print("Confusion matrix for RandomForest")
    print(tree_matrix)


def class_report(data, gnb, tree):
    gnb_report = classification_report(data.test_y, gnb.predict(data.test_x))
    print('Classification report for GaussianNB:')
    print(gnb_report)

    lda_report = classification_report(data.test_y, tree.predict(data.test_x))
    print('Classification report for RandomForest:')
    print(lda_report)


def get_data():
    raw_data = pnd.read_table('../../data/spectf.data', sep=',', header=None, lineterminator='\n')
    return Data(raw_data, 0.7)



data = get_data()

gnb = GaussianNB()
tree = RandomForestClassifier()

_ = gnb.fit(data.train_x, data.train_y)
_ = tree.fit(data.train_x, data.train_y)

В качестве оцениваемых методов классификации выберем Байесовский классификатор (GaussianNB) и дерево решений построенное с помощью алгоритма Random Forest (RandomForestClassifier).

Определим на нашем наборе данных следующие метрики:
 * Точность классификации.
 * Логарифм функции правдоподобия.
 * Область под кривой ошибок.

In [31]:
calculate_metrics(data, gnb, tree)

Accuracy for GaussianNB: 0.670 (+/- 0.07)
Accuracy for RandomTree: 0.809 (+/- 0.04)

Logarithmic loss for GaussianNB: -4.577 (+/- 1.57)
Logarithmic loss for RandomTree: -0.776 (+/- 0.47)

Area under ROC curve for GaussianNB: 0.838 (+/- 0.06)
Area under ROC curve for RandomTree: 0.775 (+/- 0.035)



Теперь рассчитаем матрицу неточностей:

In [32]:
confus_matrix(data, gnb, tree)

Confusion matrix for GaussianNB
[[16  3]
 [19 43]]
Confusion matrix for RandomForest
[[ 3 16]
 [ 4 58]]


и получим отчет классификации:

In [33]:
class_report(data, gnb, tree)

Classification report for GaussianNB:
             precision    recall  f1-score   support

          0       0.46      0.84      0.59        19
          1       0.93      0.69      0.80        62

avg / total       0.82      0.73      0.75        81

Classification report for RandomForest:
             precision    recall  f1-score   support

          0       0.43      0.16      0.23        19
          1       0.78      0.94      0.85        62

avg / total       0.70      0.75      0.71        81



# Вывод

Основываясь на полученных метриках, можно сделать вывод, что классификаторы вполне применимы для даннного датасета, но стоит задуматься о проверке других классификаторов, которые бы дали более высокие результаты на некоторых метриках.